In [1]:
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import json
import tweepy

consumer_key = "x"
consumer_secret = "x"
access_token = "213935363-x"
access_token_secret = "x"
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

def get_tweets(api, input_query):
    for tweet in tweepy.Cursor(api.search, q=input_query, lang="en").items():
        yield tweet

input_queries = ['MidtermElections2018','BlueWaveComing2018','RedWaveRising2018']
tweets = {}
for input_query in input_queries:
    tweets[input_query] = get_tweets(api, input_query)

In [2]:
from collections import defaultdict
import time

dataset = defaultdict(list)
download_tweet_count = 1000
for input_query in input_queries:
    print(input_query)
    counter = 0
    while counter < download_tweet_count:
        try:
            tweet = tweets[input_query].next()
            dataset['topic'].append(input_query)
            dataset['id'].append(tweet.id)
            # user related information
            dataset['username'].append(tweet.author.screen_name)
            dataset['name'].append(tweet.author.name)
            dataset['user_followers_count'].append(tweet.author.followers_count)
            dataset['user_friends_count'].append(tweet.author.friends_count)
            # tweet related information
            dataset['text'].append(tweet.text)
            dataset['created_at'].append(tweet.created_at.strftime("%Y-%m-%d %H:%M:%S"))
            dataset['favorite_count'].append(tweet.favorite_count)
            dataset['retweet_count'].append(tweet.retweet_count)
            # some extracted data from tweet
            dataset['hashtags'].append(
                ','.join([ht['text'] 
                          for ht in tweet.entities['hashtags']]))
            dataset['mentioned_urls'].append(
                ','.join([url['url'] 
                          for url in tweet.entities['urls']]))
            dataset['mentioned_user_ids'].append(
                ','.join([mention['id_str'] 
                          for mention in tweet.entities['user_mentions']]))
            dataset['mentioned_user_names'].append(
                ','.join([mention['screen_name'] 
                          for mention in tweet.entities['user_mentions']]))
            counter +=1
            if counter == 500:
                print "counter is now half way"
            if counter == download_tweet_count:
                print("finished collecting" + input_query)
                break
        except:
            print(len(dataset['id']))
            print('Sleeping for 15 minutes')
            time.sleep(15*60) # sleep for 15 minutes

MidtermElections2018
counter is now half way
finished collectingMidtermElections2018
BlueWaveComing2018
counter is now half way
finished collectingBlueWaveComing2018
RedWaveRising2018
counter is now half way
finished collectingRedWaveRising2018


In [153]:
import json
js = json.dumps(dataset)

# Open new json file if not exist it will create
fp = open('test.json', 'a')

# write to json file
fp.write(js)

# close the connection
fp.close()

In [67]:
import pandas as pd

dt = pd.DataFrame.from_dict(dataset)
dt[['created_at','text','hashtags','username','user_followers_count','topic']].tail()

,created_at,text,hashtags,username,user_followers_count,topic
2995,2018-04-18 17:52:30,Hell yes! #RedNationRising #RedNation #RedWave...,"RedNationRising,RedNation,RedWaveRising,redwav...",PapaMick56,211,RedWaveRising2018
2996,2018-04-18 17:52:29,RT @jojoh888: Why are you upset the people of ...,,mitchiepoo46,4562,RedWaveRising2018
2997,2018-04-18 17:52:25,RT @jojoh888: .@JerryBrownGov I'm glad Califor...,"RedWaveRising,RedWaveRising2018,EndSanctuaryCi...",DebraMMason1,26004,RedWaveRising2018
2998,2018-04-18 17:52:04,RT @jojoh888: Why are you upset the people of ...,,Thomasmccool20,134,RedWaveRising2018
2999,2018-04-18 17:50:44,"Watch out @SenDonnelly ! A big, beautiful trai...","MAGA,RedWaveRising2018",Julie121765,2140,RedWaveRising2018


In [152]:
import nltk
import string
import numpy as np

np.savetxt('xgboost.txt', dt.values, fmt='%s', delimiter="\t")  


UnicodeEncodeError: 'cp949' codec can't encode character u'\u2728' in position 198: illegal multibyte sequence

In [115]:

all_docs = dt['text'].values

import re

try:
    # Wide UCS-4 build
    myre = re.compile(u"(\ud83d[\ude00-\ude4f])|"  # emoticons
        u"(\ud83c[\udf00-\uffff])|"  # symbols & pictographs (1 of 2)
        u"(\ud83d[\u0000-\uddff])|"  # symbols & pictographs (2 of 2)
        u"(\ud83d[\ude80-\udeff])|"  # transport & map symbols
        u"(\ud83c[\udde0-\uddff])"  # flags (iOS)
        "+", flags=re.UNICODE)
except re.error:
    # Narrow UCS-2 build
    myre = re.compile(u'('
        u'\ud83c[\udf00-\udfff]|'
        u'\ud83d[\udc00-\ude4f\ude80-\udeff]|'
        u'[\u2600-\u26FF\u2700-\u27BF])+', 
        re.UNICODE)

for doc in all_docs:
    doc = myre.sub(r'', doc)
    doc.decode('unicode_escape').encode('ascii','ignore') 
    


In [133]:
from nltk.tokenize import TweetTokenizer




#exclude = set(string.punctuation)
tokenized = []
tokenizer = TweetTokenizer()
for doc in all_docs:
    tokens = tokenizer.tokenize(doc.lower())
    tokenized.append(''.join([ch for ch in ' '.join(tokens) if ch not in exclude]).split())
print(tokenized[2])

In [134]:
from nltk.corpus import stopwords

sws = set(stopwords.words('english'))
sws.add('rt') # Tweet specific stop-words
sws_removed = []
for j,sent in enumerate(tokenized):
    sws_removed.append([i for i in sent if i not in sws])
print(sws_removed[2])

In [118]:
from nltk.stem.porter import *
stemmer = PorterStemmer()
stemmerized=[]
for j,sent in enumerate(sws_removed):
    stemmerized.append([stemmer.stem(i) for i in sent ])

In [139]:
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')

import gensim

from gensim import corpora, models
dictionary = corpora.Dictionary(tokenized)

In [140]:
corpus = [dictionary.doc2bow(text) for text in tokenized]

In [109]:
print(corpus[2])

In [141]:
ldamodel = models.ldamodel.LdaModel(corpus, num_topics=3, id2word = dictionary, passes=20)

In [122]:
print(ldamodel.print_topics(num_topics=3, num_words=4))

In [123]:
print(ldamodel.print_topics(num_topics=2, num_words=4))

In [142]:
print(ldamodel.print_topics(num_topics=3, num_words=10))

In [97]:
from collections import Counter
hashtag_counter = Counter()
for hts in dt['hashtags'].values:
    hts = hts.split(',')
    hashtag_counter.update(hts)

import operator
sorted_cntr = sorted(hashtag_counter.items(), key=operator.itemgetter(1),reverse=True)
print(sorted_cntr[0:11])

In [40]:
midtermtopic_dt = dt[dt['topic'] == 'MidtermElections2018']

In [41]:
hashtag_counter_midterm = Counter()
for hts in midtermtopic_dt['hashtags'].values:
    hts = hts.split(',')
    hashtag_counter_midterm.update(hts)

import operator
sorted_cntr = sorted(hashtag_counter_midterm.items(), key=operator.itemgetter(1),reverse=True)
print(sorted_cntr[0:11])

[(u'Midterms2018', 474), (u'RedWaveRising2018', 474), (u'RedWaveRising', 474), ('', 473), (u'MidtermElections2018', 14), (u'midtermelections2018', 10), (u'farmers', 8), (u'JamesComey', 8), (u'Midterms', 8), (u'tariffs', 8), (u'ItsMuellerTime', 7)]


In [42]:
bluewavetopic_dt = dt[dt['topic'].str.contains('BlueWaveComing2018')]

In [43]:
hashtag_counter_bluewave = Counter()
for hts in bluewavetopic_dt['hashtags'].values:
    hts = hts.split(',')
    hashtag_counter_bluewave.update(hts)

import operator
sorted_cntr = sorted(hashtag_counter_bluewave.items(), key=operator.itemgetter(1),reverse=True)
print(sorted_cntr[0:11])

[('', 466), (u'BlueWaveComing2018', 293), (u'FlipItBlue', 91), (u'RedtoBlue', 89), (u'FollowBackResistance', 63), (u'Democrats', 50), (u'VoteThemOut', 22), (u'VoteBlue', 19), (u'BlueWave2018', 15), (u'FBRParty', 14), (u'bluewave', 14)]


In [44]:
redwavetopic_dt = dt[dt['topic'].str.contains('RedWaveRising2018')]

In [45]:
hashtag_counter_redwave = Counter()
for hts in redwavetopic_dt['hashtags'].values:
    hts = hts.split(',')
    hashtag_counter_redwave.update(hts)

import operator
sorted_cntr = sorted(hashtag_counter_redwave.items(), key=operator.itemgetter(1),reverse=True)
print(sorted_cntr[0:11])

[('', 528), (u'RedWaveRising2018', 265), (u'RINO', 92), (u'MAGA', 63), (u'Beto', 49), (u'RedWaveRising', 30), (u'WednesdayWisdom', 27), (u'AmericaFirst', 20), (u'Trump2020', 19), (u'BlueWave2018', 17), (u'Democrats', 17)]


In [46]:
import nltk

def format_sentence(sent):
    return({word: True for word in nltk.word_tokenize(sent)})

import sys
reload(sys)
sys.setdefaultencoding("utf-8")

post = []
with open("./pos_tweets.txt") as f:
    for i in f: 
        post.append([format_sentence(i), 'pos'])
        
negt = []
with open("./neg_tweets.txt") as f:
    for i in f: 
        negt.append([format_sentence(i), 'neg'])
        
training = post[:int((.8)*len(post))] + negt[:int((.8)*len(negt))]
test = post[int((.8)*len(post)):] + negt[int((.8)*len(negt)):]



In [47]:
from nltk.classify import NaiveBayesClassifier

classifier = NaiveBayesClassifier.train(training)

In [48]:
classifier.show_most_informative_features()

In [49]:
example1 = "Twilio is an awesome company!"

print classifier.classify(format_sentence(example1))

In [50]:
pos_count = 0
neg_count = 0
for index, row in dt.iterrows():
    if classifier.classify(format_sentence(row['text'])) == "pos":
        pos_count += 1
    elif classifier.classify(format_sentence(row['text'])) == "neg":
        neg_count += 1

print "this is pos coutn for all topic: ", pos_count
print "this is neg coutn for all topic: ", neg_count

In [51]:
pos_count = 0
neg_count = 0
for index, row in midtermtopic_dt.iterrows():
    if classifier.classify(format_sentence(row['text'])) == "pos":
        pos_count += 1
    elif classifier.classify(format_sentence(row['text'])) == "neg":
        neg_count += 1

print "this is pos coutn for midterm topic: ", pos_count
print "this is neg coutn for midterm topic: ", neg_count

In [52]:
pos_count = 0
neg_count = 0
for index, row in bluewavetopic_dt.iterrows():
    if classifier.classify(format_sentence(row['text'])) == "pos":
        pos_count += 1
    elif classifier.classify(format_sentence(row['text'])) == "neg":
        neg_count += 1

print "this is pos coutn for bluewave topic: ", pos_count
print "this is neg coutn for bluewave topic: ", neg_count

In [53]:
pos_count = 0
neg_count = 0
for index, row in redwavetopic_dt.iterrows():
    if classifier.classify(format_sentence(row['text'])) == "pos":
        pos_count += 1
    elif classifier.classify(format_sentence(row['text'])) == "neg":
        neg_count += 1

print "this is pos coutn for redwave topic: ", pos_count
print "this is neg coutn for redwave topic: ", neg_count